### Import all python libraries

In [153]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import io
import re
import bs4 as bs
import urllib.request

In [127]:
#set pandas length to full
pd.set_option('display.max_colwidth', -1)

In [128]:
#Import filling 2018
with open('index_2018.json') as f:
    d = json.load(f)

In [129]:
#create dataframe 2018
filling_2018 = json_normalize(d['Filings2018'])

In [130]:
#display top 5 rows
filling_2018.head()

,DLN,EIN,FormType,LastUpdated,ObjectId,OrganizationName,SubmittedOn,TaxPeriod,URL
0,93493318071517,453578215,990,2018-01-18T23:53:03,201713189349307151,MULEY FANATIC FOUNDATION OF WYOMING INC,2018-01-10,201612,https://s3.amazonaws.com/irs-form-990/201713189349307151_public.xml
1,93493318071467,383333202,990,2018-01-18T23:53:03,201713189349307146,KALAMAZOO COMMUNITY FOUNDATION,2018-01-10,201612,https://s3.amazonaws.com/irs-form-990/201713189349307146_public.xml
2,93493318071317,233014323,990,2018-01-18T23:53:03,201713189349307131,GOSPEL THROUGH COLOMBIA,2018-01-10,201612,https://s3.amazonaws.com/irs-form-990/201713189349307131_public.xml
3,93493318071117,351837569,990,2018-01-18T23:53:03,201713189349307111,PREMIER ARTS INC,2018-01-10,201612,https://s3.amazonaws.com/irs-form-990/201713189349307111_public.xml
4,93493318071067,133135292,990,2018-01-19T00:05:21,201713189349307106,ELDERS SHARE THE ARTS INC,2018-01-10,201706,https://s3.amazonaws.com/irs-form-990/201713189349307106_public.xml


In [131]:
#convert SubmittedOn to latest
filling_2018['DateTime'] = pd.to_datetime(filling_2018['SubmittedOn'])

In [133]:
#pick the latest filling by date
filling_2018_rdup = filling_2018.loc[filling_2018.groupby('EIN').DateTime.idxmax()]

In [134]:
filling_2018_rdup.count()

DLN                 409840
EIN                 409840
FormType            409840
LastUpdated         409840
ObjectId            409840
OrganizationName    409840
SubmittedOn         409840
TaxPeriod           409840
URL                 409840
DateTime            409840
dtype: int64

In [135]:
#read in BMF file for Missouri
ntee_filter = pd.read_csv('merge_BMF.csv'); ntee_filter.count()

EIN            15100
NAME           15100
STREET         15100
CITY           15100
ZIP            15100
ACTIVITY       15100
NTEE_CD        8546 
REVENUE_AMT    9759 
dtype: int64

In [136]:
#convert EIN to string with 9 digits
ntee_filter['EIN'] = ntee_filter['EIN'].astype(str).str.zfill(9)

In [138]:
#display top 5 rows
ntee_filter.head()

,EIN,NAME,STREET,CITY,ZIP,ACTIVITY,NTEE_CD,REVENUE_AMT
0,000360268,IGLESIA VICTORIA,1445 N BOONVILLE AVE,SPRINGFIELD,65802-1894,1002029,NaN,NaN
1,010605038,HABITAT FOR HUMANITY INTERNATIONAL INC,PO BOX 223,CASSVILLE,65625-0223,29380000,L20Z,116609.0
2,010829991,MORLEY CHURCH OF GOD,PO BOX 267,MORLEY,63767-0267,1268000,NaN,NaN
3,010905967,DIOCESE OF JEFFERSON CITY REAL ESTATE CORPORATION,PO BOX 104900,JEFFERSON CTY,65110-4900,31150120,NaN,NaN
4,010911918,MISSOURI NATIONAL EDUCATION ASSOCIATION,1810 E ELM ST,JEFFERSON CTY,65101-4100,261000000,NaN,0.0


In [139]:
#Merge 2018 filling with BMF dataset
df_merge = ntee_filter.merge(filling_2018_rdup, how='inner',on='EIN');

In [140]:
#Filter out the FormType 990PF
df_merge_990_PF = df_merge.loc[(df_merge['FormType'] != '990PF')]

In [141]:
df_merge_990_PF.count()

EIN                 2544
NAME                2544
STREET              2544
CITY                2544
ZIP                 2544
ACTIVITY            2544
NTEE_CD             1756
REVENUE_AMT         2540
DLN                 2544
FormType            2544
LastUpdated         2544
ObjectId            2544
OrganizationName    2544
SubmittedOn         2544
TaxPeriod           2544
URL                 2544
DateTime            2544
dtype: int64

In [142]:
#Select the columsn
df_columns = df_merge_990_PF[['EIN','NAME', 'STREET', 'CITY', 'ZIP', 'ACTIVITY','NTEE_CD','URL']]; df_columns.count()

EIN         2544
NAME        2544
STREET      2544
CITY        2544
ZIP         2544
ACTIVITY    2544
NTEE_CD     1756
URL         2544
dtype: int64

In [143]:
#display top 5 rows
df_columns.head()

,EIN,NAME,STREET,CITY,ZIP,ACTIVITY,NTEE_CD,URL
0,010605038,HABITAT FOR HUMANITY INTERNATIONAL INC,PO BOX 223,CASSVILLE,65625-0223,29380000,L20Z,https://s3.amazonaws.com/irs-form-990/201832499349200818_public.xml
1,015145247,AMVETS,318 BROAD ST,MOKANE,65059-1341,907000000,NaN,https://s3.amazonaws.com/irs-form-990/201833069349302108_public.xml
2,020731227,ROTARY INTERNATIONAL,3200 S STATE ROUTE 291,INDEPENDENCE,64057-1702,520349200,NaN,https://s3.amazonaws.com/irs-form-990/201811309349202611_public.xml
3,131995940,LUDHIANA CHRISTIAN MEDICAL COLLEGE BOARD U S A INC,PO BOX 1327,COLUMBIA,65205-1327,179029000,Q113,https://s3.amazonaws.com/irs-form-990/201822849349300532_public.xml
4,133352209,INTERNATIONAL PEDIATRIC NEPHROLOGY ASSOCIATION HOSPITAL,2401 GILLHAM RD,KANSAS CITY,64108-4619,161179000,NaN,https://s3.amazonaws.com/irs-form-990/201823189349312167_public.xml


In [145]:
#Filter out Null, N/A, and Blank
df_columns_na = df_columns.loc[(df_columns['URL'] != 'N/A')]
df_columns_na = df_columns_na.loc[(df_columns['URL'] != None )]
df_columns_na  = df_columns_na [df_columns_na ['URL'].notnull()]

In [146]:
df_columns_na.count()

EIN         2544
NAME        2544
STREET      2544
CITY        2544
ZIP         2544
ACTIVITY    2544
NTEE_CD     1756
URL         2544
dtype: int64

#### write a loop to pull in phone number, website, primary exempt, and description program.

In [147]:
for index, row in df_columns_na.iterrows():
    url=row["URL"]
    
    sauce = urllib.request.urlopen(url).read()
    
    soup = bs.BeautifulSoup(sauce,'xml')
    
    phone = '' if soup.find('PhoneNum') == None or soup.find('PhoneNum').text == 'N/A' else soup.find('PhoneNum').text
    df_columns_na.loc[index, 'phone'] = phone
    
    website = '' if soup.find('WebsiteAddressTxt') == None or soup.find('WebsiteAddressTxt').text == 'N/A' or soup.find('WebsiteAddressTxt').text == 'NA'else soup.find('WebsiteAddressTxt').text
    df_columns_na.loc[index, 'website'] = website
    
    primary_except = '' if soup.find('PrimaryExemptPurposeTxt') == None or soup.find('PrimaryExemptPurposeTxt').text == 'N/A' else soup.find('PrimaryExemptPurposeTxt').text
    df_columns_na.loc[index, 'primary_except'] = primary_except
    
    DescriptionProgramSrvcAccomTxt = '' if soup.find('DescriptionProgramSrvcAccomTxt') == None or soup.find('DescriptionProgramSrvcAccomTxt').text == 'N/A' else soup.find('DescriptionProgramSrvcAccomTxt').text
    df_columns_na.loc[index, 'DescriptionProgramSrvcAccomTxt'] = DescriptionProgramSrvcAccomTxt
    
    filerein = '' if soup.find('EIN') == None or soup.find('EIN').text == 'N/A' else soup.find('EIN').text
    df_columns_na.loc[index, 'filerein'] = filerein
    
    StateAbbreviationCd = '' if soup.find('StateAbbreviationCd') == None or soup.find('StateAbbreviationCd').text == 'N/A' else soup.find('StateAbbreviationCd').text
    df_columns_na.loc[index, 'StateAbbreviationCd'] = StateAbbreviationCd
    

In [151]:
#Filter out only EIN that have a website
df_columns_website_nn  = df_columns_na[df_columns_na['website'] != ""]

In [148]:
#Pull EIN with primary exempt description and description program service account.
df_columns_website_nn_aws = df_columns_website_nn[['EIN','primary_except','DescriptionProgramSrvcAccomTxt']]

In [ ]:
# Concatenate the text fields together
df_columns_website_nn_aws['value'] = df_columns_website_nn_aws['primary_except'] + df_columns_website_nn_aws['DescriptionProgramSrvcAccomTxt']

In [117]:
df_columns_website_nn_aws.count()

EIN                               1383
primary_except                    1383
DescriptionProgramSrvcAccomTxt    1383
value                             1383
dtype: int64

In [118]:
df_columns_website_nn_aws_filter = df_columns_website_nn_aws[df_columns_website_nn_aws['value'] != ""]

In [119]:
df_columns_website_nn_aws_filter.count()

EIN                               263
primary_except                    263
DescriptionProgramSrvcAccomTxt    263
value                             263
dtype: int64

In [120]:
df_columns_website_nn_aws_filter_col = df_columns_website_nn_aws_filter[['EIN','value']]

In [123]:
df_columns_website_nn_aws_filter_col.count()

EIN      263
value    263
dtype: int64

In [125]:
df_columns_website_nn_aws_filter_col.to_csv('awscomprehend.csv')

In [67]:
df_columns_website_nn_aws_filter = df_columns_website_nn_aws.loc[(df_columns_website_nn_aws['url'] != '')]
#df_columns_na_rename.loc[(df_columns['url'] != 'N/A')]

NameError: name 'df_columns_website_nn_aws' is not defined

In [95]:
df_columns_website_nn_filter = df_columns_website_nn_st[['EIN', 'NAME', 'website', 'phone', 'NTEE_CD']]

In [96]:
df_columns_website_nn_filter.dtypes

EIN        object
NAME       object
website    object
phone      object
NTEE_CD    object
dtype: object

In [97]:
df_columns_website_nn_filter.count()

EIN        1195
NAME       1195
website    1195
phone      1195
NTEE_CD    919 
dtype: int64

In [98]:
df_columns_website_nn_filter_rename = df_columns_website_nn_filter.rename(
    columns={'EIN': 'EIN', 'NAME': 'name', 'website': 'url', 'phone': 'phone','NTEE_CD': 'services' })

In [99]:
df_columns_website_nn_filter_rename.dtypes

EIN         object
name        object
url         object
phone       object
services    object
dtype: object

In [100]:
df_columns_website_nn_filter_rename.count()

EIN         1195
name        1195
url         1195
phone       1195
services    919 
dtype: int64

In [75]:
df_columns_website_nn_filter_rename.columns

Index(['EIN', 'name', 'url', 'phone', 'services'], dtype='object')

In [101]:
df_columns_website_nn_filter_rename.head()

,EIN,name,url,phone,services
2,020731227,ROTARY INTERNATIONAL,EASTERNINDEPENDENCEROTARY.ORG,8168330078,NaN
3,131995940,LUDHIANA CHRISTIAN MEDICAL COLLEGE BOARD U S A INC,HTTP://FOLUS.ORG,5737772924,Q113
5,161608735,OUR LADY OF PEACE INC,www.ladyofpeace.org,7162982900,NaN
7,201723584,LCMS NATIONAL HOUSING SUPPORT CORPORATION,WWW.NATIONALHOUSINGSUPPORT.ORG,3149961389,NaN
8,202493166,LIFEWAY CENTER INC,www.lifewaycenter.com,5732471116,NaN


In [102]:
#Change ULR to lower
df_columns_website_nn_filter_rename['url'] = df_columns_website_nn_filter_rename.url.str.lower()

In [103]:
df_columns_website_nn_filter_rename.head()

,EIN,name,url,phone,services
2,020731227,ROTARY INTERNATIONAL,easternindependencerotary.org,8168330078,NaN
3,131995940,LUDHIANA CHRISTIAN MEDICAL COLLEGE BOARD U S A INC,http://folus.org,5737772924,Q113
5,161608735,OUR LADY OF PEACE INC,www.ladyofpeace.org,7162982900,NaN
7,201723584,LCMS NATIONAL HOUSING SUPPORT CORPORATION,www.nationalhousingsupport.org,3149961389,NaN
8,202493166,LIFEWAY CENTER INC,www.lifewaycenter.com,5732471116,NaN


In [104]:
#output to csv file
df_columns_website_nn_filter_rename.to_csv('submission.csv')

In [105]:
#out ouput to json file
df_columns_website_nn_filter_rename.to_json('submission.json', orient = "records")